# जनरेटिव नेटवर्क्स

पुनरावृत्तीशील न्यूरल नेटवर्क्स (RNNs) आणि त्यांचे गेटेड सेल प्रकार जसे की लाँग शॉर्ट टर्म मेमरी सेल्स (LSTMs) आणि गेटेड पुनरावृत्तीशील युनिट्स (GRUs) यांनी भाषा मॉडेलिंगसाठी एक यंत्रणा प्रदान केली आहे, म्हणजेच ते शब्दांची क्रमवारी शिकू शकतात आणि अनुक्रमातील पुढील शब्दासाठी अंदाज देऊ शकतात. यामुळे आपल्याला RNNs **जनरेटिव कार्यांसाठी** वापरण्याची परवानगी मिळते, जसे की सामान्य मजकूर निर्मिती, मशीन अनुवाद, आणि अगदी प्रतिमेचे वर्णन तयार करणे.

आम्ही मागील युनिटमध्ये चर्चा केलेल्या RNN आर्किटेक्चरमध्ये, प्रत्येक RNN युनिटने पुढील लपवलेले स्थिती आउटपुट म्हणून तयार केले. तथापि, आपण प्रत्येक पुनरावृत्तीशील युनिटला आणखी एक आउटपुट जोडू शकतो, ज्यामुळे आपल्याला **अनुक्रम** आउटपुट करता येईल (जो मूळ अनुक्रमाच्या लांबीइतकाच असेल). याशिवाय, आपण असे RNN युनिट्स वापरू शकतो जे प्रत्येक टप्प्यावर इनपुट स्वीकारत नाहीत, आणि फक्त काही प्रारंभिक स्थिती वेक्टर घेतात, आणि नंतर आउटपुट्सचा अनुक्रम तयार करतात.

या नोटबुकमध्ये, आपण मजकूर तयार करण्यात मदत करणाऱ्या साध्या जनरेटिव मॉडेल्सवर लक्ष केंद्रित करू. सोपेपणासाठी, चला **कॅरेक्टर-लेव्हल नेटवर्क** तयार करू, जे अक्षरानुसार मजकूर तयार करते. प्रशिक्षणादरम्यान, आपल्याला काही मजकूर संग्रह घ्यावा लागेल आणि त्याला अक्षरांच्या अनुक्रमांमध्ये विभागावे लागेल.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## अक्षर शब्दसंग्रह तयार करणे

अक्षर-स्तरीय जनरेटिव्ह नेटवर्क तयार करण्यासाठी, आपल्याला मजकूर शब्दांऐवजी स्वतंत्र अक्षरांमध्ये विभागावा लागेल. हे वेगळ्या टोकनायझरची व्याख्या करून केले जाऊ शकते:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


चला पाहूया की आपण आपल्या डेटासेटमधील मजकूर कसा एन्कोड करू शकतो:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## जनरेटिव RNN प्रशिक्षण

RNNला मजकूर तयार करण्यासाठी प्रशिक्षण देण्याची पद्धत पुढीलप्रमाणे आहे. प्रत्येक चरणावर, आम्ही `nchars` लांबीचा अक्षरांचा क्रम घेऊ आणि नेटवर्कला प्रत्येक इनपुट अक्षरासाठी पुढील आउटपुट अक्षर तयार करण्यास सांगू:

!['HELLO' शब्द तयार करणाऱ्या RNNचे उदाहरण दाखवणारी प्रतिमा.](../../../../../translated_images/mr/rnn-generate.56c54afb52f9781d.webp)

खऱ्या परिस्थितीनुसार, आपल्याला काही विशेष अक्षरे समाविष्ट करायची असू शकतात, जसे की *end-of-sequence* `<eos>`. आपल्या बाबतीत, आम्हाला फक्त अखंड मजकूर तयार करण्यासाठी नेटवर्कला प्रशिक्षण द्यायचे आहे, त्यामुळे आम्ही प्रत्येक क्रमाची लांबी `nchars` टोकन इतकी निश्चित करू. परिणामी, प्रत्येक प्रशिक्षण उदाहरणामध्ये `nchars` इनपुट्स आणि `nchars` आउटपुट्स (जे इनपुट क्रम एका चिन्हाने डावीकडे सरकवलेले असतील) असतील. मिनीबॅचमध्ये अशा अनेक क्रमांचा समावेश असेल.

मिनीबॅच तयार करण्याची पद्धत अशी असेल की प्रत्येक `l` लांबीच्या बातमी मजकुरातून सर्व संभाव्य इनपुट-आउटपुट संयोजन तयार केले जातील (अशा `l-nchars` संयोजन असतील). हे एक मिनीबॅच तयार करतील, आणि प्रत्येक प्रशिक्षण चरणावर मिनीबॅचचा आकार वेगळा असेल.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

आता आपण जनरेटर नेटवर्कची व्याख्या करूया. हे कोणत्याही पुनरावृत्ती सेलवर आधारित असू शकते, जे आपण मागील युनिटमध्ये चर्चा केली होती (साधे, LSTM किंवा GRU). आपल्या उदाहरणात आपण LSTM वापरणार आहोत.

कारण नेटवर्क अक्षरे इनपुट म्हणून घेतं, आणि शब्दसंग्रहाचा आकार खूप लहान आहे, त्यामुळे आपल्याला एम्बेडिंग लेयरची गरज नाही, वन-हॉट-एनकोडेड इनपुट थेट LSTM सेलला जाऊ शकतो. मात्र, कारण आपण अक्षरांचे क्रमांक इनपुट म्हणून देतो, त्यामुळे LSTM ला पास करण्यापूर्वी त्यांना वन-हॉट-एनकोड करणे आवश्यक आहे. हे `forward` पास दरम्यान `one_hot` फंक्शन कॉल करून केले जाते. आउटपुट एनकोडर एक रेषीय लेयर असेल, जो लपलेल्या स्थितीला वन-हॉट-एनकोडेड आउटपुटमध्ये रूपांतरित करेल.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

प्रशिक्षणादरम्यान, आम्हाला तयार केलेला मजकूर नमुना स्वरूपात मिळवायचा आहे. यासाठी, आपण `generate` नावाची एक फंक्शन परिभाषित करू, जी सुरुवातीच्या स्ट्रिंग `start` पासून सुरू होऊन, लांबी `size` असलेला आउटपुट स्ट्रिंग तयार करेल.

हे कार्य पुढीलप्रमाणे चालते. प्रथम, आपण संपूर्ण `start` स्ट्रिंग नेटवर्कमधून पास करू, आणि आउटपुट स्थिती `s` आणि पुढील अंदाजित अक्षर `out` प्राप्त करू. कारण `out` हे एक-हॉट एन्कोडेड असते, त्यामुळे आम्ही `argmax` वापरून शब्दसंग्रहातील अक्षर `nc` चा अनुक्रमांक मिळवतो, आणि `itos` वापरून प्रत्यक्ष अक्षर शोधतो व ते `chars` नावाच्या अक्षरांच्या यादीत जोडतो. आवश्यक अक्षरांची संख्या तयार करण्यासाठी, हे एक अक्षर तयार करण्याचे कार्य `size` वेळा पुनरावृत्त केले जाते.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

आता प्रशिक्षण सुरू करूया! प्रशिक्षण लूप आपल्या मागील उदाहरणांप्रमाणेच आहे, परंतु अचूकतेऐवजी आम्ही प्रत्येक 1000 epochs नंतर नमुना तयार केलेला मजकूर प्रिंट करतो.

गमावलेला डेटा कसा मोजायचा याकडे विशेष लक्ष देणे आवश्यक आहे. आपल्याला एक-हॉट-एनकोडेड आउटपुट `out` आणि अपेक्षित मजकूर `text_out` दिला जातो, जो कॅरेक्टर इंडेक्सची यादी आहे, त्यावर आधारित गमावलेला डेटा मोजावा लागतो. सुदैवाने, `cross_entropy` फंक्शनला पहिला आर्ग्युमेंट म्हणून अननॉर्मलाइज्ड नेटवर्क आउटपुट आणि दुसरा आर्ग्युमेंट म्हणून वर्ग क्रमांक अपेक्षित असतो, जे आपल्याकडे आहे. हे मिनीबॅच आकारावर स्वयंचलित सरासरी देखील करते.

आम्ही `samples_to_train` नमुन्यांद्वारे प्रशिक्षण मर्यादित करतो, जेणेकरून जास्त वेळ लागणार नाही. आम्ही तुम्हाला प्रयोग करण्यासाठी प्रोत्साहित करतो आणि दीर्घ प्रशिक्षणाचा प्रयत्न करा, कदाचित अनेक epochs साठी (अशा परिस्थितीत तुम्हाला या कोडभोवती आणखी एक लूप तयार करावा लागेल).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

हे उदाहरण आधीच चांगला मजकूर तयार करते, परंतु यामध्ये काही पद्धतींनी आणखी सुधारणा करता येऊ शकते:

* **बेहतर मिनीबॅच जनरेशन**. प्रशिक्षणासाठी डेटा तयार करताना, आम्ही एका नमुन्यातून एक मिनीबॅच तयार केला. हे आदर्श नाही, कारण मिनीबॅचेस वेगवेगळ्या आकाराचे असतात, आणि काही वेळा ते तयारही होऊ शकत नाहीत, कारण मजकूर `nchars` पेक्षा लहान असतो. तसेच, लहान मिनीबॅचेस GPU पुरेशा प्रमाणात लोड करत नाहीत. सर्व नमुन्यांमधून एक मोठा मजकूर तुकडा घेणे, नंतर सर्व इनपुट-आउटपुट जोड तयार करणे, त्यांना शफल करणे, आणि समान आकाराचे मिनीबॅचेस तयार करणे अधिक शहाणपणाचे ठरेल.

* **मल्टीलायर LSTM**. LSTM सेल्सच्या 2 किंवा 3 स्तरांचा प्रयत्न करणे उपयुक्त ठरू शकते. जसे आपण मागील युनिटमध्ये नमूद केले, LSTM चा प्रत्येक स्तर मजकूरातून विशिष्ट पॅटर्न काढतो, आणि कॅरेक्टर-लेव्हल जनरेटरच्या बाबतीत आपण अपेक्षा करू शकतो की LSTM चा खालचा स्तर अक्षरसमूह (syllables) काढण्यास जबाबदार असेल, तर उच्च स्तर शब्द आणि शब्दसमूह काढतील. हे LSTM कन्स्ट्रक्टरला नंबर-ऑफ-लेयर्स पॅरामीटर पास करून सोप्या पद्धतीने अंमलात आणता येते.

* तुम्ही **GRU युनिट्स** वापरून पाहू शकता आणि कोणते चांगले कार्य करतात हे पाहू शकता, तसेच **विविध हिडन लेयर साइजेस** वापरून प्रयोग करू शकता. खूप मोठा हिडन लेयर ओव्हरफिटिंग होऊ शकतो (उदा. नेटवर्क अचूक मजकूर शिकेल), आणि लहान आकार चांगले परिणाम देऊ शकत नाही.


## मऊ मजकूर निर्मिती आणि तापमान

`generate` च्या मागील व्याख्येत, आम्ही नेहमी उच्चतम संभाव्यतेचा वर्ण पुढील वर्ण म्हणून घेत होतो, जो तयार केलेल्या मजकूरात समाविष्ट केला जात होता. यामुळे मजकूर अनेकदा त्याच वर्ण अनुक्रमांमध्ये "फिरत" राहायचा, जसे या उदाहरणात:
```
today of the second the company and a second the company ...
```

तथापि, जर आपण पुढील वर्णासाठी संभाव्यता वितरण पाहिले, तर असे होऊ शकते की काही उच्चतम संभाव्यतांमधील फरक फारसा मोठा नसतो, उदा. एका वर्णाची संभाव्यता 0.2 असू शकते, तर दुसऱ्याची 0.19, इत्यादी. उदाहरणार्थ, जर आपण '*play*' अनुक्रमात पुढील वर्ण शोधत असू, तर पुढील वर्ण जागा किंवा **e** (जसे *player* या शब्दात) असू शकतो.

यामुळे असे निष्कर्ष काढता येते की उच्च संभाव्यतेचा वर्ण निवडणे नेहमीच "योग्य" नसते, कारण दुसऱ्या क्रमांकाच्या उच्च संभाव्यतेचा वर्ण निवडल्यानेही अर्थपूर्ण मजकूर तयार होऊ शकतो. नेटवर्कने दिलेल्या संभाव्यता वितरणातून वर्ण **नमुन्याने** निवडणे अधिक शहाणपणाचे आहे.

हे नमुना निवडणे `multinomial` फंक्शन वापरून करता येते, जे तथाकथित **multinomial distribution** लागू करते. खाली दिलेली फंक्शन ही **मऊ** मजकूर निर्मिती लागू करते:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

आम्ही **temperature** नावाचा आणखी एक पॅरामीटर सादर केला आहे, जो आपण उच्चतम संभाव्यतेला किती कठोरपणे अनुसरावे हे दर्शवण्यासाठी वापरला जातो. जर temperature 1.0 असेल, तर आम्ही योग्य मल्टिनॉमियल सॅम्पलिंग करतो, आणि जेव्हा temperature अनंताकडे जाते - तेव्हा सर्व संभाव्यता समान होतात, आणि आम्ही पुढील अक्षर यादृच्छिकपणे निवडतो. खालील उदाहरणात आपण पाहू शकतो की जेव्हा आपण temperature खूप जास्त वाढवतो तेव्हा मजकूर अर्थहीन होतो, आणि जेव्हा तो 0 च्या जवळ जातो तेव्हा तो "सायकल्ड" कठोर-निर्मित मजकूरासारखा दिसतो.



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
